In [1]:
# Importem les llibreries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr # per guardar les dades 3D
from collections import Counter

In [2]:
# Carreguem les dades
data_df = pd.read_csv('results.csv')

# Emplenem els espais en blanc amb 0
data_df = data_df.fillna(0.)

In [3]:
# Read the number of games played per each player
dataarray = xr.open_dataset('stats.nc', engine='scipy')
played_games = dataarray['GamesPlayed'].isel(matchday=-1)

In [4]:
# Obtenim una llista amb tots els noms dels participants
players_names = np.unique(data_df[['Jugador 1', 'Jugador 2', 'Jugador 3', 'Jugador 4']].values.flatten())

# Llista de dies jugats
matchdays = pd.unique(data_df['D'])

In [5]:
# DataFrame with the number of team occurrences
mates_df = pd.DataFrame(columns = players_names) # number of games played with each mate
matesplayed_df = pd.DataFrame(columns = players_names) # number of games played with each mate divided by number of games playe by player
winmates_df = pd.DataFrame(columns = players_names) # number of games won by each mate

for player in players_names: # for eack player
    # Initialize a dictionary where the number of team-ups will be stored
    mate_count = {}
    mateplayed_count = {} # number of games played with mate divided by total games played
    winmate_count = {}
    for mate in players_names:
        mate_count[mate] = 0 # set counter to 0
        mateplayed_count[mate] = 0
        winmate_count[mate] = 0

    # Possible lineups
    lineups = [['Jugador 1', 'Jugador 2'], ['Jugador 2', 'Jugador 1'], ['Jugador 3', 'Jugador 4'], ['Jugador 4', 'Jugador 3']]

    # Calculate number of playing counts and victories
    for lineup in lineups:
        lineup_df = data_df[data_df[lineup[0]] == player] # pick games when player plays in position lineup[0]
        teammate_list = lineup_df[lineup[1]] # pick teammate name when player plays in position lineup[1]
        teammate_list_counts = teammate_list.value_counts() # count how many team occurences (teammate in position lineup[1])
        for mate in teammate_list_counts.keys():
            mate_count[mate] = mate_count.get(mate, 0) + teammate_list_counts.loc[mate] # store the teammate join number

            mask_mate = (teammate_list == mate) # mask selecting the games that involve 'mate'
            if lineup in [lineups[0], lineups[1]]: # pick lineups playing as local
                number_victories = (lineup_df['Local'][mask_mate] > lineup_df['Visitant'][mask_mate]).sum() # calculate number of victories
                if number_victories: # if the game was won
                    winmate_count[mate] = winmate_count.get(mate, 0) + number_victories # add number of victories
            elif lineup in [lineups[2], lineups[3]]: # pick lineups playing as visitor    
                number_victories = (lineup_df['Local'][mask_mate] < lineup_df['Visitant'][mask_mate]).sum() # calculate number of victories
                if number_victories: # if the game was won
                    winmate_count[mate] = winmate_count.get(mate, 0) + number_victories # add number of victories

    # Divide count by number of games played by player
    for mate in mate_count.keys():   
        if played_games.sel(player = player).values == 0:
            continue
        else:
            mateplayed_count[mate] = mate_count.get(mate, 0) / played_games.sel(player = player).values

    # Append this player result to the overall property dataframe
    mates_df = pd.concat([mates_df, pd.DataFrame([mate_count])])
    matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])
    winmates_df = pd.concat([winmates_df, pd.DataFrame([winmate_count])])

# Transpose to match dimesnions with xarray dimensions (x: player, y: teammate)
matesplayed_df = matesplayed_df.transpose() # necessary if matrix is not symmetric

# Set index to teammate name
mates_df = mates_df.set_index(players_names)
winmates_df = winmates_df.set_index(players_names)

# Create win / played ratio for each team (substitute 0 in the denominator by NaN, then recover 0 in the result)
winmatesplayed_df = winmates_df.div(mates_df.replace(0, pd.NA)).replace(pd.NA, 0)


C:\Users\Admin\AppData\Local\Temp\ipykernel_18632\3696880607.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])
C:\Users\Admin\AppData\Local\Temp\ipykernel_18632\3696880607.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  winmatesplayed_df = winmates_df.div(mates_df.replace(0, pd.NA)).replace(pd.NA, 0)


In [6]:
# Creem una DataArray de xarray. Hi especifiquem els noms de cada dimensió
mates_da = xr.DataArray(mates_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})
matesplayed_da = xr.DataArray(matesplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})
winmates_da = xr.DataArray(winmates_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': winmates_df.index, 'player': winmates_df.columns})
winmatesplayed_da = xr.DataArray(winmatesplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': winmatesplayed_df.index, 'player': winmatesplayed_df.columns})


# Combinem tots els DataArrays a un únic Dataset de xarray (cal que les coords siguin les mateixes per a tots)
dataset = xr.Dataset({"Teammates": mates_da,
                      "TeammatesPlayed": matesplayed_da,
                      "TeammatesWins": winmates_da,
                      "TeammatesWinsPlayed": winmatesplayed_da})

# TODO: el procés de crear el DataArray a partir del DataFrame es pot automatitzar amb una funció que faci un concat al dataframe. 

# Sote dataset
dataset.to_netcdf('teammates.nc', mode='w')

# dataset['goals'] = goals_da # si volem afegir un nou element
dataset

<xarray.Dataset> Size: 8kB
Dimensions:              (teammate: 16, player: 16)
Coordinates:
  * teammate             (teammate) object 128B 'Alejandro' 'Antía' ... 'Víctor'
  * player               (player) object 128B 'Alejandro' 'Antía' ... 'Víctor'
Data variables:
    Teammates            (teammate, player) object 2kB 0 0 0 0 0 0 ... 0 4 0 0 0
    TeammatesPlayed      (teammate, player) float64 2kB 0.0 0.0 0.0 ... 0.0 0.0
    TeammatesWins        (teammate, player) object 2kB 0 0 0 0 0 0 ... 0 2 0 0 0
    TeammatesWinsPlayed  (teammate, player) float64 2kB 0.0 0.0 0.0 ... 0.0 0.0